In [1]:
from pybtex.database.input import bibtex
import os

import pandas as pd
from nameparser import HumanName

In [2]:
df = pd.read_csv("my-talks.tsv", sep="\t", header=0,parse_dates=['date','date-start','date-end'])
display(df)
display(df.columns)

,title,key,type,conference,slides,poster,video,organization,date,date-start,...,location,url,abstract,author-01,author-02,author-03,author-04,author-05,Unnamed: 19,Unnamed: 20
0,A data-enabled wildfire smoke model,siam-pnw-2022,invited-mini-symposium,3rd Biennial Meeting of the SIAM Pacific North...,siam-pnw-2022.pdf,NaN,/files/talks/siam-pnw-2022-video.mp4,Society for Industrial and Applied Mathematics,2022-05-22,2022-05-20,...,"Vancouver, WA (USA)",https://sites.google.com/view/siampnw21/home,We report on progress towards a data enabled w...,Donna Calhoun,Jodi Mead,Patricia Azike,Sandra Babyale,NaN,NaN,NaN
1,Challenges in coupling codes in large scale PD...,icerm-2022,invited,Holistic Design of Time Dependent PDE Discreti...,icerm-2022.pdf,NaN,NaN,Institute for Computational and Experimental R...,2022-01-12,2022-01-10,...,"Providence, RI (USA)",https://icerm.brown.edu/topical_workshops/tw-2...,We discuss time stepping challenges in large s...,Donna Calhoun,Jonathan Snively,NaN,NaN,NaN,NaN,NaN
2,Simulating acoustic and gravity waves in Fores...,siam-pp-2022,invited-mini-symposium,SIAM Parallel Processing Conference,siam-pp-2022.pdf,NaN,NaN,Society for Industrial and Applied Mathematics,2022-02-26,2022-02-23,...,"Seattle, WA (USA) (virtual)",https://www.siam.org/conferences/cm/conference...,We present recent progress in the development ...,Donna Calhoun,Jonathan Snively,NaN,NaN,NaN,NaN,NaN
3,Scalable Modeling of Acoustic-Gravity Wave Int...,agu-2021,poster,Fall Meeting 2021,NaN,agu-2021.pdf,NaN,American Geophysical Union (AGU),2021-12-16,2021-12-13,...,"New Orleans, LA (USA)",https://agu.confex.com/agu/fm21/meetingapp.cgi...,Numerical simulations of acoustic-gravity wave...,Jonathan Snively,Donna Calhoun,NaN,NaN,NaN,NaN,NaN
4,ForestClaw : A parallel library for solving PD...,imr-2022,keynote,International Meshing Roundtable,imr-2022.pdf,NaN,NaN,Society for Industrial and Applied Mathematics,2022-02-24,2022-02-22,...,"Seattle, WA (USA) (virtual)",https://internationalmeshingroundtable.com/imr30/,"We describe the software package ForestClaw, a...",Donna Calhoun,Carsten Burstedde,NaN,NaN,NaN,NaN,NaN
5,"Finite volume methods, adaptive mesh refinemen...",bsu-computing-2021,colloquium,BSU Computing Colloquium,bsu-computing-2021.pdf,NaN,https://www.youtube.com/watch?v=RA23oobePqo,Boise State University,2021-09-02,NaT,...,"Boise, ID (USA) (virtual)",https://www.boisestate.edu/computing/event/don...,Many physical phenomena can be described using...,Donna Calhoun,NaN,NaN,NaN,NaN,NaN,NaN
6,Block-based logically Cartesian adaptive mesh ...,ucsb-2021,invited,Dept. of Mechanical Engineering,ucsb-2021.pdf,NaN,NaN,"University of California, Santa Barbara",2021-05-10,NaT,...,"Santa Barbara, CA (USA)",NaN,Adaptive mesh refinement (AMR) is widely seen ...,Donna Calhoun,Carsten Burstedde,Scott Aiton,NaN,NaN,NaN,NaN
7,Adaptively refined tree-based solver for the S...,siam-cse-2021,invited-mini-symposium,SIAM Computational and Engineering Conference,siam-cse-2021.pdf,NaN,NaN,Society for Industrial and Applied Mathematics,2021-03-04,2021-03-01,...,"Fort Worth, TX (USA) (virtual)",https://www.siam.org/conferences/cm/conference...,We describe our progress in developing a Serre...,Donna Calhoun,NaN,NaN,NaN,NaN,NaN,NaN
8,A fully unsplit wave propagation algorithm for...,mines-2020,invited,Dept. of Mathematics,mines-2020.pdf,NaN,NaN,Colorado School of Mines,2020-02-21,NaT,...,"Golden, CO (USA)",NaN,The focus of this talk is on a GPU implementat...,Donna Calhoun,NaN,NaN,NaN,NaN,NaN,NaN
9,Solving the Serre-Green-Naghdi equations on ad...,siam-pnw-2020,keynote,Pacific Northwest Numerical Analysis Seminar,siam-pnw-2020.pdf,NaN,https://www.youtube.com/watch?v=MglKXgPHXLo&li...,PNWNAS,2020-10-17,NaT,...,Pacific Northwest Cloud Meeting (virtual),https://sites.google.com/view/pnwnas2020/,The depth-averaged shallow water wave equation...,Donna Calhoun,NaN,NaN,NaN,NaN,NaN,NaN


Index(['title', 'key', 'type', 'conference', 'slides', 'poster', 'video',
       'organization', 'date', 'date-start', 'date-end', 'location', 'url',
       'abstract', 'author-01', 'author-02', 'author-03', 'author-04',
       'author-05', 'Unnamed: 19', 'Unnamed: 20'],
      dtype='object')

In [3]:
talk_type_list = df.type.unique()
print(talk_type_list)

['invited-mini-symposium' 'invited' 'poster' 'keynote' 'colloquium'
 'workshop' 'mini-symposium-organizer' 'tutorial']


In [4]:
# Create a map to get info that can be used in a citation.

talk_type_map = {'poster' : 'poster', \
                'colloquium' : 'colloquium', \
                'invited' : 'invited', \
                'keynote' : 'Keynote', \
                'invited-mini-symposium' : 'Mini-symposium speaker',
                'workshop' : 'Workshop lecture',
                'tutorial' : 'Tutorial',
                'mini-symposium-organizer' : "Mini-symposium organizer/speaker"}


In [5]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [6]:
# df_journals = df[df['journal'].isnull() == False]
def single_quote(s):
    return "'" + s + "'"

def double_quote(s):
    return "\"" + s + "\""

def endl():
    return "\n"

def br():
    return "<br/>"

refereed = ['article', 'inproceedings']

for row, item in df.iterrows():    
    talk_type = item['type']
    talk_id = item['key']
    citekey = talk_id.replace(':','-')
    md_filename = citekey + ".md"
    html_filename = citekey + ".html"
    # pdf_filename = item['file']
    date = item['date']
    date_start = item['date-start']
    date_end = item['date-end']
    
    year = date.year
    print(year)
    
    ## YAML variables
    
    md = "---" + endl()
    # ------
    # title: 
    # ------
    md += "title: " + single_quote(item['title']) + endl()
    
    # -----------
    # collection: 
    # -----------
    md += "collection: talks" + endl()

    
    # -----------
    # talk_type:
    # -----------
    
    talk_type_mode =  talk_type_map[talk_type]
    md += "type: " + talk_type_mode.capitalize() + endl()
    md += "talk_type: " + talk_type_mode.capitalize() + endl()
    
    # ----------
    # permalink: 
    # ----------
    # md += "permalink: /talks/" + html_filename + endl()
    md += 'permalink: /talks/' + html_filename + endl()
    
    # ---------
    # abstract:
    # ---------
    if not pd.isna(item['abstract']):
        md += "abstract: " + single_quote(html_escape(item['abstract'])) + endl()

    # ---------
    # slides:
    # ---------
    if not pd.isna(item['slides']):
        md += 'slides: ' + single_quote(item['slides']) + endl()
        
    if not pd.isna(item['poster']):
        md += 'poster: ' + single_quote(item['poster']) + endl()

    # ---------
    # video:
    # ---------
    if not pd.isna(item['video']):
        md += 'video: ' + single_quote(item['video']) + endl()
                
    # -----
    # date: 
    # -----
    
    md += "date: " + date.strftime('%Y-%m-%d') + endl()
    
    date_talk = date.strftime("%b %d, %Y")
    md += "date_talk: " + date_talk + endl()

    date_start_end = ""
    if not (pd.isna(item['date-start']) or pd.isna(item['date-end'])):
        date_start_end = "{:s} - {:s}, {:s}".format(\
                                date_start.strftime("%b %d"), \
                                date_end.strftime("%b %d"), date_end.strftime("%Y"))
        md += "date_conference: {:s}".format(single_quote(date_start_end)) + endl()
    
    
    # ------
    # venue: 
    # ------
    md += "venue: " + single_quote(item['conference']) + endl()

    # -------------
    # organization: 
    # -------------
    md += "organization: " + single_quote(item['organization']) + endl()
    
    # ---------------
    # conference_url: 
    # ---------------
    
    if not pd.isna(item['url']):
        url = item['url']
        md += "conference_url: " + single_quote(html_escape(url)) + endl()

    # ---------
    # location:
    # ---------
    md += 'location: {:s}'.format(item['location']) + endl()
        
    # ----------------------------------------
    # Authors
    # ----------------------------------------
    
    author_list = ''
    for i in range(1,6):        
        author = item['author-{:02}'.format(i)]
        if not pd.isna(author):
            a = HumanName(author)
            author_list += a.first + ' ' + a.last + ', '
            
    if author_list != '':
        author_list = author_list[:-2] + endl();
        md += 'authors: ' + author_list
            
                
    md += "---" + endl()
    
    
    print("Processed file : {:s}".format(citekey))
    print(talk_type)
    print(date_talk)
    print(author_list)
    print(talk_type_mode)
    print(date_start_end)
    print("")

    
    # ----------------------------------------
    # Markdown description for individual page
    # ----------------------------------------
    
    # Put miscellaneous content here
    
    # md += ...
    
    # ----------------------------------------
    # Write out file name
    # ----------------------------------------

    md_filename = os.path.basename(md_filename)
       
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)    
        
print("Success!")

2022
Processed file : siam-pnw-2022
invited-mini-symposium
May 22, 2022
Donna Calhoun, Jodi Mead, Patricia Azike, Sandra Babyale

Mini-symposium speaker
May 20 - May 22, 2022

2022
Processed file : icerm-2022
invited
Jan 12, 2022
Donna Calhoun, Jonathan Snively

invited
Jan 10 - Jan 14, 2022

2022
Processed file : siam-pp-2022
invited-mini-symposium
Feb 26, 2022
Donna Calhoun, Jonathan Snively

Mini-symposium speaker
Feb 23 - Feb 26, 2022

2021
Processed file : agu-2021
poster
Dec 16, 2021
Jonathan Snively, Donna Calhoun

poster
Dec 13 - Dec 17, 2021

2022
Processed file : imr-2022
keynote
Feb 24, 2022
Donna Calhoun, Carsten Burstedde

Keynote
Feb 22 - Feb 26, 2022

2021
Processed file : bsu-computing-2021
colloquium
Sep 02, 2021
Donna Calhoun

colloquium


2021
Processed file : ucsb-2021
invited
May 10, 2021
Donna Calhoun, Carsten Burstedde, Scott Aiton

invited


2021
Processed file : siam-cse-2021
invited-mini-symposium
Mar 04, 2021
Donna Calhoun

Mini-symposium speaker
Mar 01 - Mar

In [7]:
print("Done")

Done
